# Home Credit Default Risk

## Abstract:
this notebook explores the data related to home credit default risk competition in Kaggle. The data will be combined toegther and then feature exploration analysis is done. Finaly a prediction has been done and prepared to be sumbited to Kaggle

## Import Libraries

In [170]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

## load data
there are 10 files:
* 8 related to data that we need to build the model
* 1 related to data where we have to use our model to predict the labels
* 1 is an example shows how we should submit

Define global variable

In [116]:
data_path = 'the path to the data Dont forget to unzip the data'

Load file 1: application_train.csv.zip

In [117]:
df_train = pd.read_csv(data_path+'application_train.csv')

In [118]:
Y_train = df_train.TARGET
df_train = df_train.drop(['TARGET'],axis = 1)
Y_train.head()

0    1
1    0
2    0
3    0
4    0
Name: TARGET, dtype: int64

In [119]:
print(df_train.shape)
df_train.head()

(307511, 121)


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


Load file2: application_test.csv

In [120]:
df_test = pd.read_csv(data_path+'application_test.csv')
df_id_submit=df_test.SK_ID_CURR

In [121]:
print(df_test.shape)

(48744, 121)


Load file3: bureau.csv

In [122]:
df_bureau = pd.read_csv(data_path+'bureau.csv')

In [123]:
print(df_bureau.shape)
print(df_bureau.CREDIT_ACTIVE.value_counts())
df_bureau.head()

(1716428, 17)
Closed      1079273
Active       630607
Sold           6527
Bad debt         21
Name: CREDIT_ACTIVE, dtype: int64


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [124]:
df_feature_1 = df_bureau.groupby(['SK_ID_CURR'], axis=0)['DAYS_CREDIT'].agg(['count','mean','std','min','max'])
df_feature_2 = df_bureau.groupby(['SK_ID_CURR'], axis=0)['CREDIT_DAY_OVERDUE'].agg(['mean','std','min','max'])
df_feature_3 = df_bureau.groupby(['SK_ID_CURR'], axis=0)['DAYS_CREDIT_ENDDATE'].agg(['mean','std','min','max'])

df_feature_4 = df_bureau[df_bureau.CREDIT_ACTIVE=='Active'].\
groupby(['SK_ID_CURR'], axis=0)['DAYS_CREDIT'].agg(['count','mean','std','min','max'])

df_feature_5 = df_bureau[df_bureau.CREDIT_ACTIVE=='Active'].\
groupby(['SK_ID_CURR'], axis=0)['CREDIT_DAY_OVERDUE'].agg(['mean','std','min','max'])

df_feature_6 = df_bureau[df_bureau.CREDIT_ACTIVE=='Active'].\
groupby(['SK_ID_CURR'], axis=0)['DAYS_CREDIT_ENDDATE'].agg(['mean','std','min','max'])

df_feature_7 = df_bureau[df_bureau.CREDIT_ACTIVE=='Closed'].\
groupby(['SK_ID_CURR'], axis=0)['DAYS_CREDIT'].agg(['count','mean','std','min','max'])

df_feature_8 = df_bureau[df_bureau.CREDIT_ACTIVE=='Closed'].\
groupby(['SK_ID_CURR'], axis=0)['CREDIT_DAY_OVERDUE'].agg(['mean','std','min','max'])

df_feature_9 = df_bureau[df_bureau.CREDIT_ACTIVE=='Closed'].\
groupby(['SK_ID_CURR'], axis=0)['DAYS_CREDIT_ENDDATE'].agg(['mean','std','min','max'])

df_feature_10 = df_bureau[df_bureau.CREDIT_ACTIVE=='Sold'].\
groupby(['SK_ID_CURR'], axis=0)['DAYS_CREDIT'].agg(['count','mean','std','min','max'])

df_feature_11 = df_bureau[df_bureau.CREDIT_ACTIVE=='Sold'].\
groupby(['SK_ID_CURR'], axis=0)['CREDIT_DAY_OVERDUE'].agg(['mean','std','min','max'])

df_feature_12 = df_bureau[df_bureau.CREDIT_ACTIVE=='Sold'].\
groupby(['SK_ID_CURR'], axis=0)['DAYS_CREDIT_ENDDATE'].agg(['mean','std','min','max'])

df_feature_13 = df_bureau[df_bureau.CREDIT_ACTIVE=='Bad debt'].\
groupby(['SK_ID_CURR'], axis=0)['DAYS_CREDIT'].agg(['count','mean','std','min','max'])

df_feature_14 = df_bureau[df_bureau.CREDIT_ACTIVE=='Bad debt'].\
groupby(['SK_ID_CURR'], axis=0)['CREDIT_DAY_OVERDUE'].agg(['mean','std','min','max'])

df_feature_15 = df_bureau[df_bureau.CREDIT_ACTIVE=='Bad debt'].\
groupby(['SK_ID_CURR'], axis=0)['DAYS_CREDIT_ENDDATE'].agg(['mean','std','min','max'])

In [125]:
df_feature_15.describe()

,mean,std,min,max
count,20.000000,0.0,20.000000,20.000000
mean,-1632.350000,NaN,-1632.350000,-1632.350000
std,697.259695,NaN,697.259695,697.259695
min,-2558.000000,NaN,-2558.000000,-2558.000000
25%,-2168.500000,NaN,-2168.500000,-2168.500000
50%,-1748.500000,NaN,-1748.500000,-1748.500000
75%,-1055.750000,NaN,-1055.750000,-1055.750000
max,52.000000,NaN,52.000000,52.000000


Combine the features extracted from bureau.csv with train and test datasets

In [126]:
df_train.index = df_train.SK_ID_CURR
df_test.index = df_test.SK_ID_CURR

df_train = pd.concat([df_train,df_feature_1],axis=1, join_axes=[df_train.index])
df_train = pd.concat([df_train,df_feature_2],axis=1, join_axes=[df_train.index])
df_train = pd.concat([df_train,df_feature_3],axis=1, join_axes=[df_train.index])
df_train = pd.concat([df_train,df_feature_4],axis=1, join_axes=[df_train.index])
df_train = pd.concat([df_train,df_feature_5],axis=1, join_axes=[df_train.index])
df_train = pd.concat([df_train,df_feature_6],axis=1, join_axes=[df_train.index])
df_train = pd.concat([df_train,df_feature_7],axis=1, join_axes=[df_train.index])
df_train = pd.concat([df_train,df_feature_8],axis=1, join_axes=[df_train.index])
df_train = pd.concat([df_train,df_feature_9],axis=1, join_axes=[df_train.index])
df_train = pd.concat([df_train,df_feature_10],axis=1, join_axes=[df_train.index])
df_train = pd.concat([df_train,df_feature_11],axis=1, join_axes=[df_train.index])
df_train = pd.concat([df_train,df_feature_12],axis=1, join_axes=[df_train.index])
df_train = pd.concat([df_train,df_feature_13],axis=1, join_axes=[df_train.index])
df_train = pd.concat([df_train,df_feature_14],axis=1, join_axes=[df_train.index])
df_train = pd.concat([df_train,df_feature_15],axis=1, join_axes=[df_train.index])

df_test = pd.concat([df_test,df_feature_1],axis=1, join_axes=[df_test.index])
df_test = pd.concat([df_test,df_feature_2],axis=1, join_axes=[df_test.index])
df_test = pd.concat([df_test,df_feature_3],axis=1, join_axes=[df_test.index])
df_test = pd.concat([df_test,df_feature_4],axis=1, join_axes=[df_test.index])
df_test = pd.concat([df_test,df_feature_5],axis=1, join_axes=[df_test.index])
df_test = pd.concat([df_test,df_feature_6],axis=1, join_axes=[df_test.index])
df_test = pd.concat([df_test,df_feature_7],axis=1, join_axes=[df_test.index])
df_test = pd.concat([df_test,df_feature_8],axis=1, join_axes=[df_test.index])
df_test = pd.concat([df_test,df_feature_9],axis=1, join_axes=[df_test.index])
df_test = pd.concat([df_test,df_feature_10],axis=1, join_axes=[df_test.index])
df_test = pd.concat([df_test,df_feature_11],axis=1, join_axes=[df_test.index])
df_test = pd.concat([df_test,df_feature_12],axis=1, join_axes=[df_test.index])
df_test = pd.concat([df_test,df_feature_13],axis=1, join_axes=[df_test.index])
df_test = pd.concat([df_test,df_feature_14],axis=1, join_axes=[df_test.index])
df_test = pd.concat([df_test,df_feature_15],axis=1, join_axes=[df_test.index])

## Compare both train and test datasets
In this point we have to compare both train and test datastes. This is important because if there is some features that have different distribution both sets (e.g. different value range) this will lead to wrong prediction and such features have to be exculded. In order to compare both sets we follow the following plan:
1. compare the statistical properties: we will compare for each 10 variable together to make it easy to see the differences

In [127]:
## interactive tool
col_train = df_train.columns[df_train.dtypes != 'object']
col_test = df_test.columns[df_test.dtypes != 'object']
print(len(col_train))
print(len(col_test))

def f(x,print_feature=False):
    
    compare_columns(x,df_train,df_test,print_feature)


170
170


In [128]:
df_total = pd.concat([df_train,df_test])
df_total.index=range(len(df_total['SK_ID_CURR']))
for col_i in df_train.columns[df_train.dtypes == 'object']:
    
    df_total[col_i] = df_total[col_i].factorize()[0]
    df_train[col_i] = df_total.loc[range(df_train.shape[0]),col_i].values
    df_test[col_i] =  df_total.loc[range(df_train.shape[0],df_train.shape[0]+df_test.shape[0]),col_i].values

In [129]:
def compare_columns(col_i,df_train,df_test,print_feature):
    
    col_train = df_train.columns[df_train.dtypes != 'object']
    col_test = df_test.columns[df_test.dtypes != 'object']

    print('the column {} in both training and testing sets are: '.format(col_i) + 
          col_train[col_i] +' , ' + col_test[col_i])
    print('the mean value of the column {} in both training and testing sets are: '.format(col_i) + 
          str(df_train[col_train[col_i]].mean()) +' , ' + str(df_test[col_test[col_i]].mean()))
    print('the std value of the column {} in both training and testing sets are: '.format(col_i) + 
          str(df_train[col_train[col_i]].std()) +' , ' + str(df_test[col_test[col_i]].std()))
    print('the max value of the column {} in both training and testing sets are: '.format(col_i) + 
          str(df_train[col_train[col_i]].max()) +' , ' + str(df_test[col_test[col_i]].max()))
    print('the min value of the column {} in both training and testing sets are: '.format(col_i) + 
          str(df_train[col_train[col_i]].min()) +' , ' + str(df_test[col_test[col_i]].min()))
    print('Number of uniqe values of the column {}: '.format(col_i) + 
          str(len(df_train[col_train[col_i]].value_counts())) + '/' + str(len(df_train[col_train[col_i]])) + ' , ' +
          str(len(df_test[col_test[col_i]].value_counts())) + '/' + str(len(df_test[col_test[col_i]])))
    print('Number of uniqe values of the column {} in both datasets compared to the uniqe values in train set: '.format(col_i) + 
          str(len(pd.concat([df_train[col_train[col_i]],df_test[col_test[col_i]]]).value_counts())) + '/' +
         str(len(df_train[col_train[col_i]].value_counts())))
    
    print('The most frequent values of the column {} in both datasets: \n'.format(col_i) + 
          str(df_train[col_train[col_i]].value_counts().head()) + ' , \n' +
         str(df_test[col_test[col_i]].value_counts().head()))
    print ('correlation between the feature and the target')
    print(np.corrcoef([df_train[col_train[col_i]].fillna(-9999),Y_train]))
    
    if print_feature:
        
        plt.scatter(df_train[col_train[0]],df_train[col_train[col_i]],c=Y_train)


In [130]:
interact(f, x=widgets.IntSlider(min=0,max=130,step=1,value=0));

interactive(children=(IntSlider(value=0, description='x', max=130), Checkbox(value=False, description='print_feature'), Output()), _dom_classes=('widget-interact',))

## Document the non-object comparison
to delete:
['FLAG_MOBIL','FLAG_DOCUMENT_2','FLAG_DOCUMENT_4','FLAG_DOCUMENT_7','FLAG_DOCUMENT_10','FLAG_DOCUMENT_12',
'FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16',
'FLAG_DOCUMENT_17','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21']

In [131]:
df_train=df_train.drop(['SK_ID_CURR','FLAG_MOBIL','FLAG_DOCUMENT_2','FLAG_DOCUMENT_4','FLAG_DOCUMENT_7','FLAG_DOCUMENT_10','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16',
               'FLAG_DOCUMENT_17','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21'],axis=1)
df_test=df_test.drop(['SK_ID_CURR','FLAG_MOBIL','FLAG_DOCUMENT_2','FLAG_DOCUMENT_4','FLAG_DOCUMENT_7','FLAG_DOCUMENT_10','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16',
               'FLAG_DOCUMENT_17','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21'],axis=1)

In [132]:
print(df_train.shape)
print(df_test.shape)

(307511, 171)
(48744, 171)


In [133]:
df_train = df_train.fillna(-9999)
df_test = df_test.fillna(-9999)

In [134]:
from sklearn.model_selection import train_test_split

In [135]:
x_train, x_cv, y_train, y_cv= train_test_split(df_train,Y_train,
                                                  test_size=0.4,stratify=Y_train,random_state=0)

In [136]:
rf = RandomForestClassifier(max_depth=20,class_weight='balanced')
rf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=20, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [137]:
rf.score(x_train,y_train)

0.95640792169360345

In [138]:
rf.score(x_cv,y_cv)

0.8870127230600382

In [139]:
print(y_cv.value_counts())

0    113075
1      9930
Name: TARGET, dtype: int64


In [140]:
1-7448/(84806+7448)

0.9192663732737876

In [141]:
y_pred_cv=pd.DataFrame(rf.predict(x_cv))

In [142]:
y_pred_cv.sum()

0    7526
dtype: int64

In [143]:
Y_train.head()

0    1
1    0
2    0
3    0
4    0
Name: TARGET, dtype: int64

In [158]:
y_pred=pd.DataFrame(rf.predict_proba(df_test))

In [163]:
y_pred.head()

,0,1
0,0.759194,0.240806
1,0.658873,0.341127
2,0.956762,0.043238
3,0.943751,0.056249
4,0.518787,0.481213


In [164]:
Y_submit = pd.concat([df_id_submit, y_pred[1]], axis=1)

In [165]:
Y_submit.columns=['SK_ID_CURR','TARGET']

In [166]:
Y_submit.head()

,SK_ID_CURR,TARGET
0,100001,0.240806
1,100005,0.341127
2,100013,0.043238
3,100028,0.056249
4,100038,0.481213


In [167]:
import datetime, time

str(round(time.mktime((datetime.datetime.now().timetuple()))))

'1529262276'

In [168]:
Y_submit.to_csv('{}_submit.csv'.format(str(round(time.mktime((datetime.datetime.now().timetuple()))))),index=False)

In [215]:
def lgb_light(x_train,y_train,x_cv,y_cv):  

    d_train_final = lgb.Dataset(x_train, y_train)
    watchlist_final = lgb.Dataset(x_train, y_train)

    params = {
            'objective': 'binary',
            'boosting': 'gbdt',
            'learning_rate': 0.3 ,
            'verbose': 0,
            'num_leaves': 150,
            'bagging_fraction': 0.95,
            'bagging_freq': 1,
            'bagging_seed': 1,
            'feature_fraction': 0.9,
            'feature_fraction_seed': 1,
            'max_bin': 256,
            'max_depth': 10,
            'num_rounds': 2000,
            'metric' : 'auc'
        }

    #model_f1 = lgb.train(params, train_set=d_train_final,  valid_sets=watchlist_final, verbose_eval=5)
    #y_lgb_te1=model_f1.predict(x_cv)
    #err_cv_lgb1=np.divide(np.sum(np.square(np.subtract(y_lgb_te1,y_cv))),len(y_cv))

    
    params = {
        'objective': 'binary',
        'boosting': 'dart',
        'learning_rate': 0.3 ,
        'verbose': 0,
        'num_leaves': 150,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 1,
        'max_bin': 1000,
        'max_depth': 10,
        'num_rounds': 1256,
        'metric' : 'auc',
        'gpu_use_dp': True,
        'save_binary': True
    }

    model_f2 = lgb.train(params, train_set=d_train_final,  valid_sets=watchlist_final, verbose_eval=5)
    
    y_lgb_te2=model_f2.predict(x_cv)
    err_cv_lgb2=np.divide(np.sum(np.square(np.subtract(y_lgb_te2,y_cv))),len(y_cv))
    
        
    return model_f2,y_lgb_te2,err_cv_lgb2#,model_f1,err_cv_lgb1,y_lgb_te1

In [216]:
model_f2,y_lgb_te2,err_cv_lgb2= lgb_light(x_train,y_train,x_cv,y_cv)

C:\Users\Maher Deeb\AppData\Local\conda\conda\envs\tensorflow-gpu\lib\site-packages\lightgbm\engine.py:98: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[5]	valid_0's auc: 0.76163
[10]	valid_0's auc: 0.784662
[15]	valid_0's auc: 0.810623
[20]	valid_0's auc: 0.83962
[25]	valid_0's auc: 0.859663
[30]	valid_0's auc: 0.876571
[35]	valid_0's auc: 0.887325
[40]	valid_0's auc: 0.897835
[45]	valid_0's auc: 0.905509
[50]	valid_0's auc: 0.910026
[55]	valid_0's auc: 0.918245
[60]	valid_0's auc: 0.922743
[65]	valid_0's auc: 0.925875
[70]	valid_0's auc: 0.930527
[75]	valid_0's auc: 0.935361
[80]	valid_0's auc: 0.93823
[85]	valid_0's auc: 0.9398
[90]	valid_0's auc: 0.941964
[95]	valid_0's auc: 0.943558
[100]	valid_0's auc: 0.946802
[105]	valid_0's auc: 0.947847
[110]	valid_0's auc: 0.948437
[115]	valid_0's auc: 0.952413
[120]	valid_0's auc: 0.954696
[125]	valid_0's auc: 0.956306
[130]	valid_0's auc: 0.958787
[135]	valid_0's auc: 0.961108
[140]	valid_0's auc: 0.96266
[145]	valid_0's auc: 0.963553
[150]	valid_0's auc: 0.964584
[155]	valid_0's auc: 0.965608
[160]	valid_0's auc: 0.965022
[165]	valid_0's auc: 0.967708
[170]	valid_0's auc: 0.970201
[175]	

In [217]:
y_lgb_te2=model_f2.predict(df_test)
#y_lgb_te1=model_f1.predict(df_test)

print(err_cv_lgb1,err_cv_lgb2)
print((y_lgb_te1+y_lgb_te2)/2)
y_lgb_mean=(y_lgb_te1+y_lgb_te2)/2

0.0717699952961


0.0690553830321

In [211]:
Y_submit=pd.DataFrame()
Y_submit = pd.concat([df_id_submit, pd.DataFrame(y_lgb_mean)], axis=1)

In [212]:
Y_submit.columns=['SK_ID_CURR','TARGET']
Y_submit.head()
Y_submit.tail()


,SK_ID_CURR,TARGET
48739,456221,0.003951
48740,456222,0.037855
48741,456223,0.002963
48742,456224,0.008081
48743,456250,0.086402


In [213]:
Y_submit.to_csv('{}_submit.csv'.format(str(round(time.mktime((datetime.datetime.now().timetuple()))))),index=False)

In [182]:
Y_submit.TARGET = (Y_submit.TARGET+y_pred[1])/2

In [183]:
Y_submit.head()

,SK_ID_CURR,TARGET
0,100001,0.129643
1,100005,0.225948
2,100013,0.026802
3,100028,0.041935
4,100038,0.387096


In [184]:
Y_submit.to_csv('{}_submit.csv'.format(str(round(time.mktime((datetime.datetime.now().timetuple()))))),index=False)